# Working With Systems of ODE: Interacting Populations and Compartmental Models

## Example: Predator-Prey model
Consider the following population ecology model where a number of predators $P$ interacts with a number of prey $N$ (see Math 581 for more information).
$$
\begin{align*}
\frac{dN}{dt} &= rN\left(1-\frac{N}{K}\right) - cNP\\
\frac{dP}{dt} &= bNP - mP
\end{align*}
$$
where $r,K,c,b,m > 0$. For simplicity, we will non-dimensionalize this (with abuse of notation) to
$$
\begin{align*}
\frac{dx}{dt} &= x(1-x-y)\\
\frac{dy}{dt} &= b(x-a)y.
\end{align*}
$$
Since this is a 2D system, for a given parameterization, we can understand the dynamics by looking at the phase plane - that is, the vector field of $(dx/dt, dy/dt)$ in the $x,y$-plane.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# First, create a vectorized version of the ODEs given above.
# Include t, even though it doesn't appear in the equations, so that we can use this function with scipy's ODE solvers.
def f(t, xy, a, b):
    x, y = xy
    # Note: all the below operations are element-wise, so x and y can be arrays.
    dxdt = x*(1-x-y)
    dydt = b*(x-a)*y
    return np.array([dxdt, dydt])

# Now, we can use this function to compute the derivatives at a mesh of points (x, y) for given parameters a and b.
# We will use these derivatives to create a vector field plot.
a = 0.5
b = 1.0
x = np.linspace(0, 1.3, 20)
y = np.linspace(0, 1.3, 20)
X, Y = np.meshgrid(x, y)
U, V = f(0, [X, Y], a, b)
# Now we can create the vector field plot using matplotlib's quiver function.
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.quiver(X, Y, U, V, color='blue')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Vector Field of Predator-Prey System, a={}, b={}'.format(a, b))
plt.xlim(0, 1.3)
plt.ylim(0, 1.3)
plt.subplot(1, 2, 2)
# Plot it so that all arrows are the same length, to better visualize direction.
# Handle dvision by zero by adding a small epsilon to the denominator.
epsilon = 1e-10
U_norm = U/np.sqrt(U**2 + V**2 + epsilon)
V_norm = V/np.sqrt(U**2 + V**2 + epsilon)
plt.quiver(X, Y, U_norm, V_norm)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Direction Field of Predator-Prey System, a={}, b={}'.format(a, b))
plt.xlim(0, 1.3)
plt.ylim(0, 1.3)
plt.tight_layout()
plt.show()

We can analyze the behavior of solutions near equilibrium points using the Jacobian of the system, which is
$$
J = \left(
\begin{array}{cc}
1-2x-y & -x \\
by & b(x-a) 
\end{array}
\right).
$$
Plugging in equilibrium points to this Jacobian and finding eigenvalues, we can establish local stability of equations. The equilibria for this model are $(0,0)$, $(1,0)$, and $(a,1-a)$.

In [ ]:
# Define a function to compute eigenvalues of the Jacobian at a given point (x, y).
def jacobian_eigenvalues(x, y, a, b):
    # Compute the Jacobian matrix at (x, y).
    J = np.array([[1 - 2*x - y, -x],
                  [b*y, b*(x-a)]])
    # Compute the eigenvalues of the Jacobian.
    eigenvalues = np.linalg.eigvals(J) # Note: you can also get eigenvectors using np.linalg.eig(J)
    return eigenvalues

# One can show that (0,0) is always unstable. Let's print off stability for the other two matching the phase portraits above.
print("Eigenvalues at (1, 0):", jacobian_eigenvalues(1, 0, a, b))
print(f"Eigenvalues at ({a}, {1-a}):", jacobian_eigenvalues(a, 1-a, a, b))


In [ ]:
# Solve the system numerically for some initial conditions and plot solution time-series and solutions in the phase plane.
# NOTE: This cell does not depend on the previous cells, so you can change parameters a and b here to see how the solution changes.
from scipy.integrate import solve_ivp

t_span = (0, 25)
t_eval = np.linspace(t_span[0], t_span[1], 500)
IC = [0.2, 0.2] # Initial condition (x(0), y(0))
a = 0.5
b = 1.0
sol = solve_ivp(f, t_span, IC, t_eval=t_eval, args=(a, b))
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(sol.t, sol.y[0], label='Prey (x)')
plt.plot(sol.t, sol.y[1], label='Predator (y)')
plt.xlabel('Time')
plt.ylabel('Population')
plt.title('Time Series of Predator-Prey System, a={}, b={}'.format(a, b))
plt.legend()

# Now let's plot the solution in the phase plane, along with the direction field.
x = np.linspace(0, 1.3, 20)
y = np.linspace(0, 1.3, 20)
X, Y = np.meshgrid(x, y)
U, V = f(0, [X, Y], a, b)
epsilon = 1e-10
U_norm = U/np.sqrt(U**2 + V**2 + epsilon)
V_norm = V/np.sqrt(U**2 + V**2 + epsilon)
plt.subplot(1, 2, 2)
plt.quiver(X, Y, U_norm, V_norm)
plt.plot(sol.y[0], sol.y[1])
plt.xlabel('Prey (x)')
plt.ylabel('Predator (y)')
plt.title('Phase Plane with Trajectory')
plt.xlim(0, 1.3)
plt.ylim(0, 1.3)
plt.tight_layout()
plt.show()

Another option is to use pyplot's streamplot. Play with this below!

## Example: SIR model

Consider a basic epidemic compartmental model with temporary acquired immunity,
$$
\begin{align*}
\frac{dS}{dt} &= -\beta SI + \mu R\\
\frac{dI}{dt} &= \beta SI - \gamma I\\
\frac{dR}{dt} &= \gamma I - \mu R.
\end{align*}
$$

In the cell below, solve the system of equations and plot the time-series solution.

In [ ]:
# Note: you can use a dictionary to store parameters and pass them to the ODE function, 
#   which can be convenient when you have many parameters and you want to call them by name.

# parameter values
params = {}
params['beta'] = 1.4247
params['gamma'] = 0.14286
params['mu'] = 0.02

#initial values as population fractions
I0 = 1e-2
R0 = 0
S0 = 1 - I0 - R0

#time points to solve at
tpts = np.linspace(0,100,1001)

Small enough values of $R$ can cause reoccuring infection waves, but let's ignore that for now. 

Often times, you want to know how a parameter affects the long-term outcome of the model. If we assume the recovery rate, $\gamma$, and the rate at which immunity is lost, $\mu$, are constant we can vary $\beta$ between 0 and 4 and run the model in a loop to collect the final value of $I$ to compare it across this parameter sweep.

Below, give this a try.

In [ ]:
beta_list = np.linspace(0,4,1001) # For each of these, solve the system and save only the final value of I, 
                                  #   to plot against beta. This is an example of a bifurcation diagram.
                                  #   Though, you have to be careful because if there are oscillations, 
                                  #   you might just be plotting the value of I at some random point in 
                                  #   the oscillation.



## Fitting ODEs to data

We can fit numerical ODE solutions, even vector valued ones like our SIR model, 
to data using Ordinary Least Squares by taking the same approach as we did in 
the Chapter 5 lab. In this case, the model is given by solve_ivp (using the ODE 
equations) for any choice of parameter values, so you need to create a function 
that wraps around the solve_ivp routine - it will take in parameter values and 
return the solution to the ODE. 

Also, since your ODE solution and the data are often not exactly of the same 
type, you may have to have your wrapper function alter the solution of the ODE 
a bit so that it looks like your data. Examples include making sure that the 
ODE was solved at the times that correspond to you data and taking the solution 
and transforming it so that it is an apples-to-apples comparison to your data 
that can then just be subtracted inside of OLS.

Another thing you have to worry about is the relative magnitude of the different 
dimensions of your data. Often some variables are orders of magnitude bigger 
than others by nature (e.g., the number of people who die from a disease is 
often a small fraction of the estimated number of currently active cases). If 
you don't account for this, any error in the larger variable will completely 
swamp the smaller one, and OLS will only fit the model to the data that is large 
in magnitude. The solution is to minimize the square residuals of the *relative error*, 
rather than the absolute error (e.g., divide each data variable by its standard 
deviation as a normalizing factor). See "sigma" in the documentation of 
scipy.optimize.curve_fit.

### Practice

See if you can fit the SIR model above to synthetic data for the number of 
current cases and the estimated number of people who have acquired immunity 
(real data definitely wouldn't have this, but it will give you practice). The 
data can be found in Canvas as SIR_IR_data.npy. You can conduct your work below.